# Asses vegetation in the city of Munich (work in progress)

Inspired by: https://www.youtube.com/watch?v=gi4UdFsayoM

>"...the time is now right and urgent to apply space technology towards the solution of many pressing natural resources problems being compounded by population and industrial growth."

- [Stewart Udall, Secretary of the Interior, September 21, 1966](https://prd-wret.s3-us-west-2.amazonaws.com/assets/palladium/production/s3fs-public/atoms/files/1966.09.21-DOI-Udall-Earth%20Resources%20Studied%20From%20Space.pdf)

## Landsat

Landsat is the first "civilian Earth observation satellite" launched in 1972, a collaboration between the Department of the Interior, NASA, and the Department of Agriculture.

At over 40 years, the Landsat series of satellites provides the longest temporal record of moderate resolution multispectral data of the Earth’s surface on a global basis. The Landsat record has remained remarkably unbroken, proving a unique resource to assist a broad range of specialists in managing the world’s food, water, forests, and other natural resources for a growing world population. It is a record unmatched in quality, detail, coverage, and value. Source: USGS

- [NASA's Landsat page](https://www.nasa.gov/mission_pages/landsat/overview/index.html)
- [USGS's Landsat page](https://www.usgs.gov/core-science-systems/nli/landsat)

## How to access landsat data

- [USGS Earth Explorer](https://earthexplorer.usgs.gov/)
- [Amazon AWS Open Data](https://registry.opendata.aws/landsat-8/)
- [Google Earth Engine](https://developers.google.com/earth-engine/datasets/catalog/landsat-8)

### The Workflow

In this lab, we will do the following:

- Use Google Earth Engine's Python API
- define an AOI (area of interest)
- import multiple Landsat images in summer (for highest vegetation)
- determine which images are best for analysis
- create various NDVI map outputs to assess the vegetation

## Python libraries in this workshop

- [pandas](https://pandas.pydata.org/docs/getting_started/index.html)
- [geopandas](https://geopandas.org/en/stable/gallery/index.html)
- [google earth engine](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)


## Import libraries

In [1]:
# the regulars
import pandas as pd
import geopandas as gpd

# earth engine
import ee

# allow images to display in the notebook
from IPython.display import Image

## Authenticate Earth Engine

In [2]:
# Trigger the authentication flow.
# ee.Authenticate()

# Initialize the library.
ee.Initialize()

# Check setup (should print "USGS Landsat 8 Level 2, Collection 2, Tier 1")
print(ee.ImageCollection("LANDSAT/LC08/C02/T1_L2").get("title").getInfo())

USGS Landsat 8 Level 2, Collection 2, Tier 1


For this lab, we will use Google Earth Engine's "USGS Landsat 8 Level 2, Collection 2, Tier 1"

- [EE Landsat 8](https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C02_T1_L2)

You can also:

- `.filterBounds()` method allows you to filter by location
- `.filterDate()` allows you to filter by date

## Define filters

In [76]:
# coordinates of München
lat = 48.13  #39.444012
lon = 11.56  #-121.833619

# point of interest as an ee.Geometry
poi = ee.Geometry.Point(lon,lat)
sb1 = [11.550510928753445, 48.14317046659895, 11.548483995100966, 48.14362920069638, 11.548425207444321, 48.143508757625455, 11.548538746909973, 48.14342179836262, 11.54857563229527, 48.143385253685224, 11.548617804710055, 48.14334351325487, 11.548534475022251, 48.14284241658684, 11.54856398535273, 48.142817614078396, 11.548555711389987, 48.14276622692296, 11.548500661695646, 48.142426491566866, 11.548466015460622, 48.142211884901805, 11.548450588128363, 48.14211676993932, 11.548440381592137, 48.14211753356593, 11.54843390183638, 48.142077312613054, 11.548421829234211, 48.14200367505895, 11.548417713923211, 48.14197818524611, 11.548231027671909, 48.14084644704717, 11.548242143846183, 48.140844755366174, 11.548229086611222, 48.14078455824207, 11.548223287796656, 48.14078107241928, 11.548193815162467, 48.14076337228245, 11.54814219270692, 48.14044672762327, 11.548134261297623, 48.140417197800865, 11.548124155224944, 48.14039044709342, 11.548110013736029, 48.140364490444924, 11.548092109994203, 48.14033879001628, 11.548067627592854, 48.140312884604306, 11.54806560830459, 48.140313236228955, 11.547829403655868, 48.140130489252414, 11.54776480751931, 48.1400802316544, 11.54755282489168, 48.13991565264225, 11.547478504464243, 48.13985736035578, 11.54732463319864, 48.13973517974746, 11.547193807550961, 48.13960759929762, 11.547166119990504, 48.13957898351086, 11.547170298639161, 48.139577200134106, 11.547139297437143, 48.13954353530052, 11.547134995475423, 48.139543968121586, 11.547009665027938, 48.13940723347788, 11.546967081220918, 48.1393930414216, 11.54698289509054, 48.13926754480837, 11.547329162743438, 48.139211018969064, 11.548608776098444, 48.13901447825759, 11.549790264156758, 48.13885357385316, 11.549748304436793, 48.13768035915724, 11.549726766804717, 48.13713703340019, 11.549480187161963, 48.1365703954018, 11.548416510339791, 48.13575927851087, 11.548282706060325, 48.135699223797396, 11.548159468581346, 48.135629954563484, 11.548043134087697, 48.13555511980331, 11.547936526816637, 48.13547431226684, 11.547837711608036, 48.13538903617207, 11.547750093237152, 48.13529839270523, 11.547653857867754, 48.13517981724322, 11.547571952541956, 48.13505671102907, 11.547506075009755, 48.134929265885596, 11.547433148844508, 48.13475603535432, 11.547368829087533, 48.134581398279934, 11.547344118371038, 48.134499594471535, 11.547315540714603, 48.13440500396434, 11.547284892005047, 48.13428355823932, 11.547272351408356, 48.13422702876288, 11.547217959918404, 48.133959697784675, 11.547188081631555, 48.13378088297139, 11.547169834124402, 48.13360205496244, 11.547163484386804, 48.133422173978595, 11.547161793650627, 48.133242212165285, 11.547155349390263, 48.133119058988754, 11.5471388234093, 48.13302734774155, 11.547102765397641, 48.132895191985156, 11.547046794893665, 48.13276547442829, 11.546976021773187, 48.132639022017116, 11.546885992786745, 48.132518194654104, 11.546782083153074, 48.13240279932627, 11.546662798951973, 48.132294235907146, 11.546530964623681, 48.13219245702528, 11.546433129440565, 48.13210665140681, 11.546351636292018, 48.13201377454769, 11.546288064816371, 48.1319147569427, 11.546240396935316, 48.13180833955686, 11.546230120983637, 48.131786873320046, 11.546193443538131, 48.13165261648419, 11.546187356939386, 48.131565169940885, 11.546228002739035, 48.13138506497457, 11.546294447342008, 48.13121202551167, 11.546366043605008, 48.131038281528845, 11.546391014414299, 48.130965653507864, 11.546397878194458, 48.13093347225983, 11.546439054382388, 48.130800844785064, 11.546618950711249, 48.13048599082973, 11.546358820015302, 48.13034135478761, 11.545569522244444, 48.12948186139586, 11.545505400600293, 48.12938007337089, 11.545466100929733, 48.12930023636985, 11.545333975781444, 48.1290455577834, 11.545290421693611, 48.12894177969134, 11.545260171454487, 48.12883580370179, 11.545114556207364, 48.128147077668714, 11.545066040316293, 48.12792045789836, 11.5450475818598, 48.127835213660816, 11.544997684607742, 48.127657386134125, 11.544994541467718, 48.127648289571745, 11.544981563445818, 48.12761172154905, 11.544828885908249, 48.12718260285471, 11.544803842617847, 48.127112260202246, 11.544768340962618, 48.12702325177207, 11.544726231225024, 48.126937457137274, 11.544671903185034, 48.1268534277454, 11.544610309211022, 48.12677110245027, 11.54459995823616, 48.12675694675421, 11.544606855111471, 48.12675375444992, 11.544592760201276, 48.12673775883744, 11.544586147717316, 48.126739091263204, 11.544554116776057, 48.126700846074634, 11.544527518292936, 48.12667089360741, 11.544503570328757, 48.12664526711478, 11.544482943753506, 48.126624270263584, 11.544465958817366, 48.12660709514364, 11.54444410394253, 48.126585284188465, 11.544420949247435, 48.1265626535177, 11.544397675824174, 48.126540584244985, 11.544377801985865, 48.12652230697545, 11.544353331481691, 48.126500354191776, 11.54433205039791, 48.12648161230143, 11.544310266820933, 48.12646276456509, 11.544292190627061, 48.12644716573909, 11.54427096258188, 48.12642925137348, 11.544250792008324, 48.126412664171205, 11.544225666845637, 48.12639271890751, 11.544197827907734, 48.12637094598219, 11.544211300733426, 48.12631319654849, 11.54430195388717, 48.126241260882345, 11.544365739164531, 48.1262301031129, 11.544459072680963, 48.12620503674009, 11.544443533309652, 48.12612285631055, 11.544715899690452, 48.12610177760644, 11.54488138304305, 48.1261013665508, 11.545393793337468, 48.125879064863554, 11.545490146788303, 48.125871608089824, 11.545494321303751, 48.12588574492888, 11.54557312687761, 48.12584635995451, 11.545662427456602, 48.12580175655139, 11.545747034904908, 48.125759711740244, 11.54581719087621, 48.12572661712143, 11.547217201953144, 48.12505250325346, 11.54849647307316, 48.12444144997436, 11.548506626007745, 48.12444814501849, 11.548663186739153, 48.12437393899687, 11.548820282094079, 48.12429837929985, 11.548837277443955, 48.12429053204346, 11.548899573955461, 48.12426424682119, 11.54903198894966, 48.124200637648535, 11.549139236494229, 48.12431404577336, 11.549854953811622, 48.12396963881548, 11.550260154539021, 48.12377156114771, 11.55074195452618, 48.123519967976954, 11.552560044522961, 48.122611048753996, 11.55405680295005, 48.121891048376476, 11.55424623354774, 48.1218019867926, 11.55497147850622, 48.12146108595731, 11.55646791449237, 48.120741528782396, 11.557053109008304, 48.12046007980681, 11.55829692557348, 48.11985433285048, 11.559356225459748, 48.11935499372411, 11.559410742743703, 48.119327772927136, 11.559480620123809, 48.11929428824274, 11.559565540066018, 48.119253613629716, 11.559630665547049, 48.119222291687656, 11.559653724015845, 48.11921131765943, 11.559655688851542, 48.11921318683813, 11.559923942136365, 48.11908589568238, 11.559926936123965, 48.119084571033746, 11.559905254788637, 48.11904585373491, 11.560712446884374, 48.11867247042525, 11.560772861444459, 48.11864475217003, 11.560838630935928, 48.11863411140433, 11.560845088230534, 48.11863287597287, 11.560851805334764, 48.11863281168254, 11.560858379021747, 48.11863391672646, 11.560864135859372, 48.11863618722455, 11.560868811159654, 48.118639353740846, 11.560872136914407, 48.118643323273574, 11.560987839997056, 48.11858825460567, 11.561352821538565, 48.11841429650129, 11.561396473570866, 48.11839342308065, 11.561385944505904, 48.11838384920924, 11.561385312654718, 48.11837916995915, 11.56138669459957, 48.11837458796596, 11.561389685806166, 48.118370372292766, 11.561394417043779, 48.11836688264125, 11.561400215860322, 48.11836429687136, 11.561408275944915, 48.11836405503068, 11.561416191272261, 48.118365252493895, 11.561423288913407, 48.118367706313364, 11.561429166985858, 48.11837141465446, 11.561433561209922, 48.11837592721429, 11.561699859625381, 48.11824916763961, 11.561720696531003, 48.11823925861221, 11.561957682035583, 48.11812436281249, 11.561949027542274, 48.11811551920263, 11.561947451236744, 48.1181114665863, 11.561947620444464, 48.11810733184322, 11.561949535965043, 48.11810329311539, 11.561953056662789, 48.11809988983009, 11.561958048974613, 48.11809730042695, 11.561963835574462, 48.11809579375752, 11.561970015879501, 48.11809536706, 11.561976185316865, 48.1180962885007, 11.56198167328836, 48.118098286903084, 11.561992879164233, 48.11810695932262, 11.562514443365322, 48.11785591996849, 11.562575460751018, 48.117910725700376, 11.562650007254307, 48.11797768002305, 11.562659366463745, 48.117983022941864, 11.562666623799375, 48.11798242100922, 11.562669288458935, 48.11798503934945, 11.562739733907241, 48.11797757471367, 11.562748266868581, 48.11798543209923, 11.563022138098663, 48.118237234096306, 11.563624027263506, 48.11880742185144, 11.564090140923653, 48.11924397566255, 11.564200259453703, 48.11933451752841, 11.564443580201111, 48.11953281243247, 11.564542864653975, 48.11960969850961, 11.564688337028919, 48.119709448130784, 11.56482739816722, 48.11980333459575, 11.564976098676173, 48.119894919889205, 11.565037036875028, 48.11993103730014, 11.565288002670506, 48.12006919090359, 11.56597636445997, 48.120446265833145, 11.566444782069961, 48.12070204249893, 11.566724288136395, 48.12085130297788, 11.567454230799207, 48.121240708243455, 11.567862782956933, 48.12146023736169, 11.5680312916748, 48.12155093314712, 11.568194833937387, 48.1216356184202, 11.568336414214507, 48.12169353162278, 11.568393178259914, 48.121716678248305, 11.56854961131064, 48.121798240437585, 11.568699489644043, 48.121876385815305, 11.568761956183131, 48.121908907480154, 11.568765338239825, 48.12190604246689, 11.568775905303225, 48.121911567527896, 11.568772523291623, 48.121914433440246, 11.568839400440236, 48.121949700675266, 11.568904269491316, 48.12198390873536, 11.568907649272266, 48.12198113284873, 11.56891821238385, 48.12198665888489, 11.568914832603133, 48.12198943477182, 11.568945980666859, 48.12200609966532, 11.569040220829502, 48.12205644990689, 11.56910249144756, 48.122089722601416, 11.569277968827624, 48.12218344146972, 11.569326869297837, 48.12221582270292, 11.56948678204928, 48.12232218697031, 11.569596012498566, 48.12237952664378, 11.569730981639688, 48.12245022639599, 11.569861661536367, 48.12251863930023, 11.571050603501659, 48.12314131679705, 11.572209525411262, 48.123768693553664, 11.572355326258938, 48.123839642542144, 11.572506220091283, 48.12392714151729, 11.572546477870938, 48.12394904157181, 11.572791467419203, 48.12408894674736, 11.57301215787655, 48.12420487809212, 11.573807950529703, 48.124621959163235, 11.57383898561047, 48.12463853078921, 11.574629814762023, 48.125056033251965, 11.574657101780401, 48.12507007296599, 11.57521766498559, 48.12536708980883, 11.57524949782597, 48.12538430361957, 11.575281198053933, 48.125401337736136, 11.575313701617608, 48.125418095589396, 11.575345394166787, 48.125435029083214, 11.576461253502416, 48.126025467047306, 11.577239675939273, 48.126439200645, 11.577419208182969, 48.12653760666636, 11.577605808427778, 48.1266416871716, 11.57761048414566, 48.126650785363665, 11.577602847235877, 48.12665893975522, 11.57761259596371, 48.12667935332156, 11.577602381277293, 48.126699667837514, 11.577612806999651, 48.12670608876001, 11.5777137459351, 48.126768471134945, 11.577817544492715, 48.12683153787104, 11.577828766397143, 48.12683850507724, 11.577821333529203, 48.12684405089036, 11.577828016812171, 48.126847852798576, 11.577846593332934, 48.1268587157187, 11.57784779007337, 48.1268600691475, 11.57801365083424, 48.126961691936465, 11.578178968913864, 48.127049620957195, 11.578324371200234, 48.12712221258083, 11.57843586315371, 48.12717441376401, 11.578570676898547, 48.12723281633546, 11.578631305793177, 48.12725984016126, 11.578698767372515, 48.127286883310184, 11.578759402931668, 48.12731111304855, 11.578841191032412, 48.12734342398962, 11.578885942162213, 48.127360096597656, 11.578892870762248, 48.12736267551029, 11.57895675467147, 48.127387005868826, 11.579022001769133, 48.12740981281841, 11.57916257102675, 48.12745591120287, 11.579414391711243, 48.12753649354746, 11.579647708095987, 48.127608885290094, 11.579978231027857, 48.127710995368815, 11.58016942634871, 48.127776428391485, 11.580289282611293, 48.127823261548414, 11.580355285228146, 48.12785229626327, 11.580488072774688, 48.12791360706906, 11.580498234695803, 48.12790357276573, 11.580592140779293, 48.12795276304011, 11.58058128079547, 48.127966755047986, 11.580629026294481, 48.127991204134425, 11.580634776679911, 48.12799419182908, 11.580731731988019, 48.12804489622137, 11.580804971295168, 48.12808320334871, 11.580830604844405, 48.12809660750773, 11.580836090026096, 48.12809950577306, 11.580856708318855, 48.12810839316376, 11.580878276151713, 48.12811611377189, 11.580887924173195, 48.128119206427634, 11.58089240751555, 48.12811292587505, 11.580946312143022, 48.12812534740695, 11.580983741964756, 48.12813150492566, 11.58103114329423, 48.1281338301647, 11.58120166410795, 48.128128325355036, 11.58126724088243, 48.12812622713202, 11.582192276580281, 48.128096301064076, 11.58229439613153, 48.128093527351986, 11.582395008794569, 48.12809407169044, 11.58249412658035, 48.12809686035513, 11.582594166004716, 48.12810242748038, 11.58269444588878, 48.12811150102086, 11.582792814630087, 48.12812389721234, 11.582892385413507, 48.128137748254204, 11.582990192026283, 48.128154287888385, 11.583085832087797, 48.12817342529145, 11.583181154698675, 48.12819643978378, 11.583274577918454, 48.128221161543, 11.5833667754631, 48.12824821337668, 11.583456958760486, 48.12827856681069, 11.583545106065326, 48.12831152049687, 11.58362987652482, 48.1283459060718, 11.583712840940825, 48.12838345630913, 11.583794043454024, 48.12842306886765, 11.583872409018664, 48.128464560082996, 11.583949973467302, 48.128508457330824, 11.584024049789427, 48.12855422000083, 11.584094747451768, 48.1286018600336, 11.584162861559136, 48.12865192821804, 11.584226773192443, 48.12870414335572, 11.584254211493384, 48.12872806527086, 11.584287585010838, 48.128757150742594, 11.585068802274913, 48.12945874612228, 11.585078802993852, 48.12946759369181, 11.585104044080152, 48.12949118514342, 11.585143053577104, 48.129527642427796, 11.585149586306063, 48.12953375438133, 11.585163316558601, 48.12954658328458, 11.585404466183478, 48.12976206001598, 11.585527493315892, 48.12986923505235, 11.585651292240541, 48.12997570963643, 11.58577642600599, 48.1300811066717, 11.585903575743359, 48.1301864095938, 11.586031035666544, 48.130290946862466, 11.586159888848814, 48.13039423159372, 11.58629031520931, 48.13049797204604, 11.586422070137074, 48.13060000161146, 11.586555096269041, 48.13070065000334, 11.586689608758366, 48.13080211237651, 11.586772928641327, 48.13086364730788, 11.586785074775877, 48.13085649390194, 11.586858923004943, 48.130909900969414, 11.586845553912507, 48.13091885002919, 11.587082638434278, 48.13109462576561, 11.587158470623445, 48.13115649733771, 11.587202340496676, 48.131198194639595, 11.587237374062754, 48.131235183476, 11.587270258514941, 48.131271719526005, 11.587272004372398, 48.131271636108366, 11.58728557316239, 48.13128805138933, 11.587315332726575, 48.13133033913955, 11.587342814973864, 48.13137307319384, 11.587368143819223, 48.13141633894251, 11.58735899822446, 48.131417833475304, 11.587369040007433, 48.13143999602293, 11.587424512584343, 48.131567625256956, 11.586530121451856, 48.131741565346715, 11.585724348717346, 48.13196331260453, 11.585258085210496, 48.132184905406106, 11.583536017482416, 48.132975452142816, 11.583354693522633, 48.133212756163466, 11.582837930493806, 48.1338959287167, 11.582760787421511, 48.13387043198423, 11.58223900795676, 48.13370249876769, 11.582165920969775, 48.13368828297835, 11.582097362124333, 48.13367979152975, 11.582024936628583, 48.13367680373986, 11.581954829663447, 48.13367834015058, 11.581885140684644, 48.13368282295465, 11.581542050416857, 48.13372454979692, 11.580928642817758, 48.13379870429156, 11.579977125405613, 48.1339410061585, 11.579892954829766, 48.13395103815865, 11.57981249951908, 48.13395720750324, 11.579717065009268, 48.13396102733436, 11.579631324186728, 48.13396086552359, 11.579512092915762, 48.13395317046727, 11.579425537109053, 48.13394404233816, 11.579320027510441, 48.133927014867346, 11.579176615814694, 48.133896871772585, 11.578353861649685, 48.133697980287245, 11.577623233798914, 48.13352231234676, 11.576982310210557, 48.133367574678076, 11.576354788282933, 48.13321765087385, 11.576290624328792, 48.13320498040317, 11.576226617134658, 48.13319643464617, 11.576142820094624, 48.13318940078049, 11.576047013789415, 48.1331886423408, 11.575941588671824, 48.13319465366151, 11.57584967725263, 48.13320607822733, 11.57503916222268, 48.13331382076946, 11.574965425813996, 48.133318463894156, 11.574899182877859, 48.13331410028357, 11.574837952338378, 48.13330334099189, 11.57478334913325, 48.13328711080723, 11.57473768147328, 48.13326678507324, 11.574682522134342, 48.133232691956245, 11.574295325416198, 48.1329520620483, 11.573807527065597, 48.13259891937185, 11.573667162226185, 48.13249930255651, 11.573599930561027, 48.13245627091774, 11.573516496446535, 48.13240724161682, 11.573449422069464, 48.13236966273039, 11.573082919761731, 48.13217649586712, 11.572976705856066, 48.13211815971996, 11.572442782816054, 48.13183514797846, 11.572116740511879, 48.13166469511103, 11.571818991618914, 48.13151293811628, 11.571519344993918, 48.13135652140629, 11.571233960599937, 48.13120681654625, 11.570925547905768, 48.131036716078455, 11.570845757385166, 48.130995305544886, 11.570762583295515, 48.1309590239437, 11.570689610660816, 48.1309328787095, 11.570607435719028, 48.13090769071584, 11.570552384438633, 48.13089344450984, 11.570482957142833, 48.13088087870428, 11.570416798402858, 48.130873984907396, 11.570342494481878, 48.130869650612446, 11.570263035354685, 48.13087120022579, 11.570188241356403, 48.1308774817954, 11.56987732844674, 48.13094902163778, 11.56844660618066, 48.13135581477381, 11.568294991311005, 48.13140163047801, 11.568188335463983, 48.131435988683464, 11.568121720238763, 48.13146235662688, 11.568063189553367, 48.13149038835303, 11.567979820278417, 48.13153983086239, 11.567942953537099, 48.131566611535476, 11.567898389425263, 48.131600865232876, 11.567824083909482, 48.131665838816026, 11.567532809532345, 48.13193347381162, 11.567368678535587, 48.1320849827387, 11.5671221094515, 48.13230900214084, 11.56708134419999, 48.13234894459498, 11.567052722175921, 48.13238810995028, 11.567032309774735, 48.132429827194976, 11.567308694853711, 48.13265045495658, 11.567093851495938, 48.13264903125998, 11.566650667711192, 48.1333750478782, 11.566158132191385, 48.13412134621729, 11.565150694735303, 48.135715808364104, 11.56486303646947, 48.13631484091752, 11.564832594598053, 48.13637822741027, 11.564982024843998, 48.1375584187537, 11.565018845230911, 48.13785082975818, 11.56523975157038, 48.139103706022354, 11.565663229786026, 48.13964449088761, 11.561438204199254, 48.14096790955136, 11.561125752530172, 48.141049045077814, 11.56073757747673, 48.14115826971143, 11.556842778579504, 48.141996748113, 11.556162332590784, 48.14205003417064, 11.555771480889447, 48.14208245639887, 11.550510928753445, 48.14317046659895]
poi = ee.Geometry.Polygon(sb1)

# start date of range to filter for
# clouds impact quality, so we need to chose between a variety of images
start_date = '2022-05-01'

# end date
end_date = '2022-08-31'

## Get Landsat 8 data

In [77]:
# get the satellite data
landsat = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")\
            .filterBounds(poi)\
            .filterDate(start_date,end_date)\
            .filter(ee.Filter.lt('CLOUD_COVER', 0.5)) # Filter by 'CLOUD_COVER', ie. only return images with CLOUD_COVER less than 3 (3%)

In [78]:
# how many images did we get?
print('Total number:', landsat.size().getInfo())

Total number: 1


In [1]:
# information about the first image in our collection
landsat.first().getInfo()

In [79]:
image = landsat.first()
clipped_image = image.clip(poi.buffer(1000))
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }
display(Image(url = clipped_image.getThumbUrl(parameters)))

In [18]:
# what about cloud cover of our first image?
# Number is in percentage
landsat.first().get('CLOUD_COVER').getInfo()

12.36

In [19]:
# when was this image taken?
landsat.first().get('DATE_ACQUIRED').getInfo()

'2022-06-28'

## Bands

[Landsat 8 Bands](https://www.usgs.gov/media/images/landsat-8-band-designations)

In [75]:
# what bands did we get?
landsat.first().bandNames().getInfo()

['SR_B1',
 'SR_B2',
 'SR_B3',
 'SR_B4',
 'SR_B5',
 'SR_B6',
 'SR_B7',
 'SR_QA_AEROSOL',
 'ST_B10',
 'ST_ATRAN',
 'ST_CDIST',
 'ST_DRAD',
 'ST_EMIS',
 'ST_EMSD',
 'ST_QA',
 'ST_TRAD',
 'ST_URAD',
 'QA_PIXEL',
 'QA_RADSAT']

In [43]:
# put the images in a list
landsat_list = landsat.toList(landsat.size())

## Display satellite image

- [What are the min/max values?](https://gis.stackexchange.com/questions/304180/what-are-the-min-and-max-values-of-map-addlayer-on-google-earth-engine) <-- Read! (Not so easy to set, roughly speaking, they control brightness)

In [44]:
# set some parameters for the images
parameters = {
                'min': 7000,
                'max': 16000,
                'dimensions': 800, # square size in pixels
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'] # bands to display (r,g,b)
             }

In [45]:
landsat.size().getInfo()

1

In [46]:
# create an empty data container
data = []

# loop through each image and display it
# `landsat.size().getInfo()` gives number of images in list
for i in range(landsat.size().getInfo()):
    # singe Image-object
    image = ee.Image(landsat_list.get(i))
    
    # when was this image taken?
    date = image.get('DATE_ACQUIRED').getInfo()
    
    # cloud cover
    cloud = image.get('CLOUD_COVER').getInfo()

    # print the image info
    print('Image #', i, date, 'Cloud cover:', cloud)
    
    # display the image
    display(Image(url = image.getThumbUrl(parameters)))

    # data to list
    this_data = [i, date, cloud]

    # append the data 
    data.append(this_data)
    

# Create the pandas DataFrame
df = pd.DataFrame(data, columns = ['Image #', 'Date', 'Cloud Cover'])

Image # 0 2022-07-18 Cloud cover: 0.3


In [47]:
# our data in a dataframe
df

,Image #,Date,Cloud Cover
0,0,2022-07-18,0.3


## Selecting images, zooming in

Now that we have inspected our collection of images, we can pick and choose which ones are relevant for our study, ie. the image having the lowest cloud cover.

We also want to create an ROI (region of interest) and zoom in to the area of interest. We do so by appying a 20km buffer around our POI.

In [48]:
# create a list of images we want (before, during, after)
landsat_sequence = [0]

In [52]:
# Define a region of interest with a buffer zone of 3 km
# Initially, the POI is for selecting which images, the ROI controls which radius around the point is displayed, ie. the zoom
roi = poi.buffer(10000) # meters

In [57]:
parameters = {
                'min': 6000,
                'max': 16000,
                'dimensions': 800,
                'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
                'region': roi
             }

In [58]:
for i in landsat_sequence:

    image = ee.Image(landsat_list.get(i))   
    # when was this image taken?
    date = image.get('DATE_ACQUIRED').getInfo()

    # cloud cover
    cloud = image.get('CLOUD_COVER').getInfo()
    
    print('Image #',i,date,'Cloud cover:',cloud)
    
    display(Image(url = image.getThumbUrl(parameters)))

Image # 0 2022-07-18 Cloud cover: 0.3


## Normalized Difference Vegetation Index (NDVI)

The normalized difference vegetation index (NDVI) is a simple graphical indicator that can be used to analyze remote sensing measurements, often from a space platform, assessing whether or not the target being observed contains live green vegetation.

https://www.agricolus.com/en/vegetation-indices-ndvi-ndmi/

### Calculating NDVI in Google Earth Engine

https://developers.google.com/earth-engine/tutorials/tutorial_api_06

In [59]:
# ndvi palette: red is low, green is high vegetation
palette = ['red', 'yellow', 'green']

ndvi_parameters = {'min': 0,
                   'max': 0.4,
                   'dimensions': 512,
                   'palette': palette,
                   'region': roi}

In [60]:
for i in landsat_sequence:

    image = ee.Image(landsat_list.get(i))

    # when was this image taken?
    date = image.get('DATE_ACQUIRED').getInfo()
    
    # print some information
    print('Image #', i, date)
    
    # display the image

    display(Image(url=image.normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)))

Image # 0 2022-07-18


In [61]:
import requests

image = ee.Image(landsat_list.get(0))
# Multi-band GeoTIFF file.
url = image.normalizedDifference(['SR_B5', 'SR_B4']).getThumbUrl(ndvi_parameters)
response = requests.get(url)
with open('multi_band', 'wb') as fd:
  fd.write(response.content) 

## Folium

Of course, we can't end the lab without an interactive map. For this, we can use folium, which uses the open-source javascript mapping library leaflet.

    https://python-visualization.github.io/folium/quickstart.html

Google earth engine works with folium:

    https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard?hl=en#interactive_mapping_using_folium

Currently, I am not able to add the cropped image of munich to the folium map.

In [62]:
# a simple folium map
import folium

m = folium.Map(location=[lat,lon])
# m  # Show the map

In [63]:
# Google function that allows ee layers on folium
def add_ee_layer(self, ee_image_object, vis_params, name):
    """Adds a method for displaying Earth Engine image tiles to folium map."""
    map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
    folium.raster_layers.TileLayer(
        tiles=map_id_dict['tile_fetcher'].url_format,
        attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
        name=name,
        overlay=True,
        control=True
    ).add_to(self)

# Add Earth Engine drawing method to folium
folium.Map.add_ee_layer = add_ee_layer

In [74]:
# Create a map
my_map = folium.Map(location=[lat, lon], zoom_start=10)

# Add a layer for each satellite image of interest (before, during and after)
for i in landsat_sequence:

    # when was this image taken?
    date = ee.Image(landsat_list.get(i)).get('DATE_ACQUIRED').getInfo()

    my_map.add_ee_layer(ee.Image(landsat_list.get(i)).normalizedDifference(['SR_B5', 'SR_B4']), 
                        ndvi_parameters, 
                        name=date)
    
# Add a layer control panel to the map
folium.LayerControl(collapsed = False).add_to(my_map)

# Display the map.
display(my_map)

# TODO Add cropped images of Munich to folium

## Save the folium map as an html file

In [90]:
my_map.save('camp.html')

# Resources

- [Google's Python Tutorial](https://developers.google.com/earth-engine/tutorials/community/intro-to-python-api-guiattard)
- [Earth Lab](https://www.earthdatascience.org/courses/use-data-open-source-python/multispectral-remote-sensing/landsat-in-Python/)
    